In [1]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import ApexTrainer
from training.configs import GENERAL_CONFIG
from training.platforms.traffic_env import TrafficEnv
import numpy as np
from time import sleep

/home/trey/anaconda3/envs/traffic_sim/lib/python3.8/site-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Model(object):
    "class to load checkpoints and test them"
    def __init__(self, cp_path: str) -> None:
        self.config = GENERAL_CONFIG.copy()
        self.config['num_workers'] = 0
        self.config['in_evaluation'] = True
        self.config['explore'] = False
        
        self.agent = ApexTrainer(self.config, env=TrafficEnv)
        self.agent.restore(cp_path)
        self.policy = self.agent.workers.local_worker().get_policy('traffic_light')

    def act(self, obs):
        actions = {}
        for ob in obs:
            actions[ob] = self.policy.compute_single_action(np.array(obs[ob]).reshape(12,))[0]
        # print(actions)
        return actions
    
    def create_env(self):
        return TrafficEnv(self.agent.config['env_config'])



In [3]:
my_model = Model(cp_path='tune_data/testing_tune/APEX_TrafficEnv_0701d_00000_0_2022-02-16_20-02-22/checkpoint_000026/checkpoint-26')

2022-02-16 20:18:55,337	WARNING trainer_template.py:185 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!
2022-02-16 20:18:55,338	WARNING util.py:57 -- Install gputil for GPU system monitoring.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


2022-02-16 20:18:55,433	INFO trainable.py:416 -- Restored on 192.168.0.206 from checkpoint: tune_data/testing_tune/APEX_TrafficEnv_0701d_00000_0_2022-02-16_20-02-22/checkpoint_000026/checkpoint-26
2022-02-16 20:18:55,433	INFO trainable.py:424 -- Current state after restoring: {'_iteration': 26, '_timesteps_total': 13312, '_time_total': 829.524355173111, '_episodes_total': 44}


In [4]:
env = my_model.create_env()

In [5]:
obs = env.reset()
dones = {'__all__': False}

total_reward = 0
while not dones['__all__']:
    env.render()
    sleep(0.05)
    actions = my_model.act(obs)
    obs, rewards, dones, info = env.step(actions)
    total_reward += sum(list(rewards.values()))

print('Episode Complete')
print('Total Reward: {}'.format(total_reward))

[[0. 0. 0. 0.]
 [1. 1. 0. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 0. 1.]]
Added 14.0 lights...
Found 20 roads


KeyboardInterrupt: 